In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [2]:
!pip install pydrive                             # Package to use Google Drive API - not installed in Colab VM by default
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth                    # Other necessary packages
from oauth2client.client import GoogleCredentials
auth.authenticate_user()                         # Follow prompt in the authorization process
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
your_module = drive.CreateFile({"id": "176fVmFMju81nPM2jsNOADdU5YnEA6Cl7"})   # "your_module_file_id" is the part after "id=" in the shareable link
your_module.GetContentFile("dlc_practical_prologue.py")          # Save the .py module file to Colab VM
import dlc_practical_prologue as prologue

In [4]:
N = 1000
train_input, train_target, train_classes, test_input, test_target, test_classes = prologue.generate_pair_sets(N)

Extracting ./data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to ./data/mnist/MNIST/raw


Extracting ./data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/mnist/MNIST/raw



Extracting ./data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/mnist/MNIST/raw


Extracting ./data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/mnist/MNIST/raw
Processing...
Done!


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


In [0]:
train_input_first_approach =  torch.zeros(size = (1000,392))
test_input_first_approach = torch.zeros(size = (1000,392))

for i in range(train_input.size(0)):
    #print(torch.cat((train_input[i,0],train_input[i,1]),0).size())
    train_input_first_approach[i]  = torch.cat((torch.flatten(train_input[i,0]),torch.flatten(train_input[i,1])),0)
    test_input_first_approach[i] = torch.cat((torch.flatten(test_input[i,0]),torch.flatten(test_input[i,1])),0)
    


In [6]:
import torch
import torch.nn.functional as F
from torch import nn,optim

# define the network class
class MyNetwork(nn.Module):
    def __init__(self):
        # call constructor from superclass
        super().__init__()
        
        # define network layers
        self.fc1 = nn.Linear(392, 100)
        self.fc2 = nn.Linear(100, 20)
        self.fc3 = nn.Linear(20, 1)
        self.fc4 = nn.Linear(100, 20)
        
    def forward(self, x):
        # define forward pass
        x = F.relu(self.fc1(x))
        y = F.relu(self.fc4(x))
        x = F.relu(self.fc2(x))
        y = y.view(-1,2,10)
        x = torch.sigmoid(self.fc3(x))
        
        return x,y

# instantiate the model
model = MyNetwork()

# print model architecture
print(model)

MyNetwork(
  (fc1): Linear(in_features=392, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=20, bias=True)
  (fc3): Linear(in_features=20, out_features=1, bias=True)
  (fc4): Linear(in_features=100, out_features=20, bias=True)
)


In [7]:
import torch
import torch.nn.functional as F
from torch import nn,optim

# define the network class
class MyNetworkv2(nn.Module):
    def __init__(self):
        # call constructor from superclass
        super().__init__()
        
        # define network layers
        self.fc1 = nn.Linear(392, 100)
        self.fc2 = nn.Linear(100, 20)
        self.fc3 = nn.Linear(20, 1)
       
        
    def forward(self, x):
        # define forward pass
        x = F.relu(self.fc1(x))
        y = F.relu(self.fc2(x))
        x = F.relu(self.fc2(x))
        y = y.view(-1,2,10)
        x = torch.sigmoid(self.fc3(x))
        
        return x,y

# instantiate the model
model = MyNetwork()

# print model architecture
print(model)

MyNetwork(
  (fc1): Linear(in_features=392, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=20, bias=True)
  (fc3): Linear(in_features=20, out_features=1, bias=True)
  (fc4): Linear(in_features=100, out_features=20, bias=True)
)


In [0]:
def training_model(train_input, train_target, model, batch, lr):
    optimizer = torch.optim.SGD(model.parameters(), lr = lr)
    criterion = nn.BCELoss()
    total_loss = 0
    
    for b in range(0, train_input.size(0), batch):
        output, aux = model(train_input.narrow(0, b, batch))            
        loss = criterion(output, train_target.narrow(0, b, batch))
        model.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss

In [0]:
def compute_nb_errors(model, data_input, data_target, batch):
    
    nb_data_errors = 0

    for b in range(0, data_input.size(0), batch):
        output, _ = model(data_input.narrow(0, b, batch))
        
        predicted_classes = (output>0.5).float()
        
        for k in range(batch):
            if data_target[b + k].item() != predicted_classes[k,0].item():
                nb_data_errors = nb_data_errors + 1
    return nb_data_errors

In [12]:
epochs = 100
losses = torch.zeros((3, epochs))
model = MyNetwork()
for e in range(epochs):
    print("loss", training_model(train_input_first_approach, train_target.float(), model , 100, 0.001))
    print(compute_nb_errors(model, train_input_first_approach,train_target, 100), " ",compute_nb_errors(model, test_input_first_approach,test_target, 100))
    

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:516: UserWarning: Using a target size (torch.Size([100])) that is different to the input size (torch.Size([100, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


loss 17.433743178844452
412   450
loss 8.481733322143555
352   382
loss 6.785908937454224
305   363
loss 5.968097031116486
275   343
loss 5.459536582231522
237   326
loss 5.088926047086716
221   312
loss 4.792184412479401
204   302
loss 4.5539257526397705
196   292
loss 4.348800987005234
187   286
loss 4.174935400485992
178   283
loss 4.016602277755737
170   277
loss 3.875290185213089
163   275
loss 3.7442402243614197
156   274
loss 3.6249408423900604
150   270
loss 3.5192261934280396
146   268
loss 3.4117250740528107
137   270
loss 3.3156998455524445
133   265
loss 3.226496398448944
129   264
loss 3.1375719606876373
129   264
loss 3.057205468416214
124   258
loss 2.980749785900116
119   251
loss 2.9071027040481567
111   252
loss 2.8368504643440247
106   247
loss 2.764714300632477
100   248
loss 2.697783038020134
99   244
loss 2.63492988049984
95   244
loss 2.5709518790245056
91   245
loss 2.508291721343994
88   245
loss 2.451751336455345
85   243
loss 2.391588643193245
83   247
loss 2

In [0]:
def training_aux(train_input, train_target, train_classes, model, batch, lr):
    optimizer = torch.optim.SGD(model.parameters(), lr = lr)
    Binary_Criterion = nn.BCELoss()
    Aux_Criterion = nn.CrossEntropyLoss()
    total_loss_aux = 0
    total_loss_bin = 0
    final_total_loss = 0
    for b in range(0, train_input.size(0), batch):
        output, aux = model(train_input.narrow(0, b, batch))
        target_classes = train_classes.narrow(0, b, batch)
        target_comparison = train_target.narrow(0, b, batch)
        aux_loss = Aux_Criterion(aux[:,0], target_classes[:,0]) + Aux_Criterion(aux[:,0], target_classes[:,1])
        binary_loss = Binary_Criterion(output, target_comparison)
        final_loss = 0.9*binary_loss + 0.1*aux_loss
        model.zero_grad()
        final_loss.backward()
        optimizer.step()
        total_loss_aux += aux_loss
        total_loss_bin += binary_loss
        final_total_loss += final_loss
    return final_total_loss, total_loss_aux, total_loss_bin

In [0]:
def compute_nb_errors_aux(model, data_input, data_target, batch):
    
    nb_data_errors = 0

    for b in range(0, data_input.size(0), batch):
        output, _ = model(data_input.narrow(0, b, batch))
        predicted_classes = (output>0.5).float()
        for k in range(batch):
            if data_target[b + k] != predicted_classes[k]:
                nb_data_errors = nb_data_errors + 1
    return nb_data_errors

In [15]:
epochs = 170
losses = torch.zeros((3, epochs))
model2 = MyNetwork()
for e in range(epochs):
    print("loss",training_aux(train_input_first_approach, train_target.float(), train_classes, model2 , 100, 0.001)[2].item())
    print(compute_nb_errors_aux(model2, train_input_first_approach,train_target, 100), " ",compute_nb_errors_aux(model2, test_input_first_approach,test_target, 100))

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:516: UserWarning: Using a target size (torch.Size([100])) that is different to the input size (torch.Size([100, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


loss 13.169427871704102
382   395
loss 8.563200950622559
316   334
loss 7.00161075592041
292   314
loss 6.25323486328125
270   300
loss 5.709685325622559
250   291
loss 5.317044258117676
232   288
loss 4.997866153717041
224   278
loss 4.744796276092529
204   272
loss 4.535516738891602
192   274
loss 4.352724552154541
181   269
loss 4.188638210296631
173   268
loss 4.048004150390625
167   270
loss 3.916588306427002
160   269
loss 3.799027919769287
149   267
loss 3.6916399002075195
138   265
loss 3.5883538722991943
132   265
loss 3.4980196952819824
127   261
loss 3.408651113510132
124   262
loss 3.3286032676696777
122   258
loss 3.2463412284851074
117   259
loss 3.1734023094177246
113   257
loss 3.0985703468322754
110   257
loss 3.033933639526367
108   257
loss 2.9668869972229004
107   256
loss 2.9057023525238037
104   253
loss 2.8429207801818848
102   254
loss 2.781179428100586
101   255
loss 2.726832389831543
98   254
loss 2.6714913845062256
91   250
loss 2.6134233474731445
90   250
lo